In [ ]:
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/snkd93bnjr-1.zip

--2024-10-21 20:08:58--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/snkd93bnjr-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.92.1.122, 52.218.102.96, 3.5.66.213, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.92.1.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278867361 (266M) [application/zip]
Saving to: ‘snkd93bnjr-1.zip’

snkd93bnjr-1.zip    100%[===================>] 265.95M  23.8MB/s    in 12s     

2024-10-21 20:09:12 (21.5 MB/s) - ‘snkd93bnjr-1.zip’ saved [278867361/278867361]



In [ ]:
!unzip /content/snkd93bnjr-1.zip

Archive:  /content/snkd93bnjr-1.zip
  inflating: A dataset for microscopic peripheral blood cell images for development of automatic recognition systems/PBC_dataset_normal_DIB.zip  


In [ ]:
#RENAME "A dataset for microscopic peripheral blood cell images for development of automatic recognition systems"
# To "PBC"
import os
old = '/content/A dataset for microscopic peripheral blood cell images for development of automatic recognition systems'
new = "/content/PBC"
os.rename(old,new)

In [ ]:
!unzip /content/PBC/PBC_dataset_normal_DIB.zip

Streaming output truncated to the last 5000 lines.
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_292483.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_935950.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_890863.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_519370.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_215077.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_388154.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_968896.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_34345.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_349907.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_245683.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_785122.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_615525.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_714613.jpg  
  inflating: PBC_dataset_normal_DIB/erythroblast/ERB_904757.jpg  
  inflating: PBC_dataset_n

In [ ]:
#this file is corrupted !
!rm -rf /content/PBC_dataset_normal_DIB/neutrophil/.DS_169665.jpg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/WBC/PBC_PREP.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/PBC_prepd/eosinophil/3955_7.jpg  
  inflating: content/PBC_prepd/eosinophil/3527_1.jpg  
  inflating: content/PBC_prepd/eosinophil/1751_0.jpg  
  inflating: content/PBC_prepd/eosinophil/4089_1.jpg  
  inflating: content/PBC_prepd/eosinophil/4041_0.jpg  
  inflating: content/PBC_prepd/eosinophil/4027_0.jpg  
  inflating: content/PBC_prepd/eosinophil/2334_7.jpg  
  inflating: content/PBC_prepd/eosinophil/2263_2.jpg  
  inflating: content/PBC_prepd/eosinophil/2977_1.jpg  
  inflating: content/PBC_prepd/eosinophil/4126_2.jpg  
  inflating: content/PBC_prepd/eosinophil/2006_0.jpg  
  inflating: content/PBC_prepd/eosinophil/2688_4.jpg  
  inflating: content/PBC_prepd/eosinophil/2930_0.jpg  
  inflating: content/PBC_prepd/eosinophil/2763_1.jpg  
  inflating: content/PBC_prepd/eosinophil/3088_0.jpg  
  inflating: content/PBC_prepd/eosinophil/3441_1.jpg  
  inflating: content/PBC_prepd/eosinophil/4288_2.jpg  
  inflating: c

In [ ]:
!pip install -q "monai-weekly[pillow, tqdm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.5 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import tempfile
import matplotlib.pyplot as plt
import PIL
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import classification_report

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch, DataLoader
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121,ViT
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    Resize
)
from monai.utils import set_determinism

print_config()

MONAI version: 1.5.dev2443
Numpy version: 1.26.4
Pytorch version: 2.5.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: d508ba780862a6459326c812e09188838c25bdc5
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.13.1
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.20.0+cu121
tqdm version: 4.66.5
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.44.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.i

In [ ]:
#classes are constrained to basophil, eosinophil, lymphocyte, monocyte , neutrophil

classes= ['basophil','eosinophil','lymphocyte','monocyte','neutrophil']
base_dir = "/content/content/PBC_prepd"
num_class = len(classes)
image_files = [
    [os.path.join(base_dir, classes[i] +"/" + x) for x in os.listdir(os.path.join(base_dir, classes[i]))]
    for i in range(num_class)
]
num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)

In [ ]:
index = 1000
test_img = PIL.Image.open(os.path.join(classes[image_class[index]], image_files_list[index]))
print(test_img.size)

(149, 161)


In [ ]:
import torch
from torch.utils.data import Dataset
class PBC_Dataset(Dataset):
  def __init__(self,img_files,lbls,transform):
    self.imgs =img_files
    self.lbls = lbls
    self.transform = transform
  def __len__(self):
    return len(self.imgs)
  def __getitem__(self,index):
    if self.transform:
      return self.transform(self.imgs[index]), self.lbls[index]


In [ ]:
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
        Resize((180,180))
    ]
)

val_transform = Compose([LoadImage(image_only=True), EnsureChannelFirst(),ScaleIntensity(),Resize((180,180))])
y_pred_trans = Compose([Activations(softmax=True)])
y_trans = Compose([AsDiscrete(to_onehot=num_class)])

In [ ]:
train_ds = PBC_Dataset(image_files_list,image_class,train_transforms)
train_loader = DataLoader(train_ds,num_workers = 4,batch_size = 50)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =DenseNet121(
      spatial_dims = 2,
      in_channels = 3,
      out_channels = len(classes),
      init_features=64,
      growth_rate=32,
      block_config=(6, 12, 24, 16),
      pretrained=True,
      progress=False,
).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3,weight_decay = 1e-5)
max_epochs = 15
val_interval = 1
auc_metric = ROCAUCMetric()

In [ ]:
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, " f"train_loss: {loss.item():.4f}")
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

----------
epoch 1/15
1/218, train_loss: 1.6671
2/218, train_loss: 1.3269
3/218, train_loss: 1.0734
4/218, train_loss: 0.8673
5/218, train_loss: 0.6979
6/218, train_loss: 0.5936
7/218, train_loss: 0.5037
8/218, train_loss: 0.4310
9/218, train_loss: 0.3740
10/218, train_loss: 0.3320
11/218, train_loss: 0.3093
12/218, train_loss: 0.2648
13/218, train_loss: 0.2398
14/218, train_loss: 0.2199
15/218, train_loss: 0.2074
16/218, train_loss: 0.1920
17/218, train_loss: 0.1729
18/218, train_loss: 0.1708
19/218, train_loss: 0.1427
20/218, train_loss: 0.1479
21/218, train_loss: 0.1345
22/218, train_loss: 0.1241
23/218, train_loss: 0.1154
24/218, train_loss: 0.1234
25/218, train_loss: 0.1143
26/218, train_loss: 2.2030
27/218, train_loss: 3.8105
28/218, train_loss: 3.1020
29/218, train_loss: 2.3892
30/218, train_loss: 1.8403
31/218, train_loss: 1.3794
32/218, train_loss: 1.0472
33/218, train_loss: 0.7940
34/218, train_loss: 0.6256
35/218, train_loss: 0.5003
36/218, train_loss: 0.4530
37/218, train_l

In [ ]:
torch.save(model.state_dict(),'DenseNet121_SGD.pt')

In [ ]:
!cp DenseNet121_SGD.pt /content/drive/MyDrive/WBC

In [ ]:
test_file = 'test.png'
test_ds = val_transform(test_file)
test_dl= DataLoader(test_ds)


In [ ]:
y_pred_trns =Compose([
    Activations(softmax=True)
])

In [ ]:
tst = test_ds.to(device)
with torch.no_grad():
  model.eval
  out = model(tst.unsqueeze(0))
  y_pred_act = [y_pred_trns(i) for i in decollate_batch(out.cpu())]

  y_prd = y_pred_act
  print(classes[np.argmax(y_prd)])

neutrophil
